In [1]:
import pandas as pd
import numpy as np

## Load combined datasets with demography and clinical scales

[combined](https://docs.google.com/spreadsheets/d/1Sry9B78TtzQZIxZsHlgVUOApPfdON2lGIhKYj2Nt3pc/edit#gid=391698543)

[pd data](https://docs.google.com/spreadsheets/d/1nS2YOswtcSKBBsAnKnxFsng9hzg5IV6IkiVzMITD99Q/edit#gid=635915857)

[psychiatric participant data](https://docs.google.com/spreadsheets/u/0/d/1kSkywJC18Dtytk7yNNLgWRq_e8AVdfkB/edit?usp=sheets_home&ths=true&rtpof=true)

[demography](https://docs.google.com/spreadsheets/d/1GIuIjkeAFOoTP3n9Q_1-eFtYkbFKmvFBIlpHrIKM7tA/edit#gid=1966721918)

norm - separate csv

______

0 - при поступлении, 

1 - при выписке и на момент осмотра, 

2 - через год

Hdrs просто наверно соответствует тому, который 1


при различиях в HDRS за основу брать те значения, которые в таблицах 0, 1 и 2
 
при различиях в пансс - взять значения из панс-форм

 
диагноз брать из демографии, https://docs.google.com/spreadsheets/d/1GIuIjkeAFOoTP3n9Q_1-eFtYkbFKmvFBIlpHrIKM7tA/edit?usp=sharing там они написаны словами, но в целом понятно, первый - это первый депрессивный эпизод, F32

In [2]:
dataframes = pd.read_excel('/Users/galina.ryazanskaya/Downloads/thesis?/2023_annotation/combined_scales_2023_annotation.xlsx', sheet_name=None)

In [3]:
dataframes.keys()

dict_keys(['patient_data', 'PD_participant_data', 'combined_old_data_scales', 'demography', 'sans-1-total', 'sans-2-total', 'hdrs-0-total', 'hdrs-total', 'hdrs-1-total', 'hdrs-2-total', 'panss', 'panss-', 'saps-ftd-1-total', 'saps-ftd-2-total', 'sops', 'sops-1-total', 'sops-2-total'])

In [4]:
dataframes['PD_participant_data'].set_index('ID').head(2)

,longitudinal,group,diagnosis,sex,age,education.level,education.years,depression.symptoms,dep.scale,thought.disorder.symptoms,td.scales,2.depression.symptoms,2.thought.disorder.symptoms
ID,,,,,,,,,,,,,
PD-001,NaN,patient,schizotypal.disorder,female,19.0,secondary,11,1,HDRS,0,SAPS,NaN,NaN
PD-002,NaN,patient,bipolar.affective.disorder,female,26.0,higher,17,1,HDRS,0,SAPS,NaN,NaN


In [5]:
dataframes['combined_old_data_scales'].set_index('ID').head(2)

,oldID,diagnosis,hdrs-total-,dep.severity-,panss-td--,panss-total--,panss-o-total--,panss-p-total--,panss-n-total--,sex,age,educ.years
ID,,,,,,,,,,,,
PN-001,H01,NaN,NaN,NaN,4,30,16,7,7,m,27.0,22
PN-002,H02,NaN,1.0,0.0,6,33,17,8,8,f,36.0,17


In [6]:
dataframes['demography'].rename(columns={'id': 'ID'}).set_index('ID').head(2)

,group,ds,age,educ.years,edulevel,socstat1,socstat2,age-onset,GAF1,Condition-at-discharge,CPE1,exacerbation,rehospitalization,GAF2,CPE2
ID,,,,,,,,,,,,,,,
PD-002,1,БАР,26,17,4,2,NaN,14,10,2,587.0,1.0,0.0,NaN,NaN
PD-004,1,ПРЛ,16,9,0,1,1.0,14,20,1,800.0,1.0,0.0,55.0,200.0


In [7]:
dataframes['patient_data'].set_index('ID').head(2)

,oldID,group,diagnosis.rus,diagnosis,sex,age,educ.years,ID.HDRS,DEP.SEVERITY(0-3),HDRS,...,SOPS Pos Total,SOPS Neg Total,SOPS Des Total,SOPS Com Total,PANSS.ID,panss 1 sum,panss p sum,panss n sum,panss o sum,panss TD
ID,,,,,,,,,,,,,,,,,,,,,
PD-001,PD01,NaN,Шизотипическое,F21.3,f,19.0,11.0,PD-001,1.0,12.0,...,15.0,15.0,9.0,9.0,PD-001,NaN,NaN,NaN,NaN,NaN
PD-002,PD02,dep,БАР,F31.4,f,26.0,17.0,PD-002,1.0,13.0,...,1.0,5.0,2.0,16.0,PD-002,43.0,7.0,10.0,26.0,4.0


### merge old data to demography

In [8]:
x = pd.merge(dataframes['combined_old_data_scales'].set_index('ID'), dataframes['demography'].rename(columns={'id': 'ID'}).set_index('ID'), on='ID', how='outer')
x.head(2)

,oldID,diagnosis,hdrs-total-,dep.severity-,panss-td--,panss-total--,panss-o-total--,panss-p-total--,panss-n-total--,sex,...,socstat1,socstat2,age-onset,GAF1,Condition-at-discharge,CPE1,exacerbation,rehospitalization,GAF2,CPE2
ID,,,,,,,,,,,,,,,,,,,,,
PN-001,H01,NaN,NaN,NaN,4.0,30.0,16.0,7.0,7.0,m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PN-002,H02,NaN,1.0,0.0,6.0,33.0,17.0,8.0,8.0,f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
def merge_cols(row, colname_1, colname_2, resolve, verbose=True):
    col_1, col_2 = row[colname_1], row[colname_2]
    if pd.isnull(col_1) and pd.isnull(col_2):
        return col_1
    elif not pd.isnull(col_1) and pd.isnull(col_2):
        return col_1
    elif pd.isnull(col_1) and not pd.isnull(col_2):
        return col_2
    else:
        if col_1 == col_2:
            return col_1
        else:
            if verbose:
                print(row)
            res = resolve(col_1, col_2)
            return res

In [10]:
x['age'] = x.apply(lambda row: merge_cols(row, 'age_x', 'age_y', lambda x, y: x), axis=1)

In [11]:
x['educ.years'] = x.apply(lambda row: merge_cols(row, 'educ.years_x', 'educ.years_y', lambda x, y: x), axis=1)

In [12]:
x.drop(columns=['age_x', 'age_y', 'educ.years_x', 'educ.years_y'], inplace=True)

In [13]:
x.rename(columns={'group': 'group.code'}, inplace=True)

In [14]:
cols_test = [c for c in ['diagnosis.type',
 'diagnosis_code',
 'diagnosis_eng',
 'diagnosis_rus',] if c in x.columns] 
if not cols_test:
    cols_test = x.columns
x.loc['PD-019'][cols_test]

oldID                       NaN
diagnosis                   NaN
hdrs-total-                 NaN
dep.severity-               NaN
panss-td--                  NaN
panss-total--               NaN
panss-o-total--             NaN
panss-p-total--             NaN
panss-n-total--             NaN
sex                         NaN
group.code                  2.0
ds                          ПРЛ
edulevel                    3.0
socstat1                    0.0
socstat2                    0.0
age-onset                  15.0
GAF1                       10.0
Condition-at-discharge      1.0
CPE1                      562.0
exacerbation                1.0
rehospitalization           1.0
GAF2                       25.0
CPE2                      250.0
age                        22.0
educ.years                 13.0
Name: PD-019, dtype: object

### merge to patient data

In [15]:
x = pd.merge(x, dataframes['patient_data'].set_index('ID'), on='ID', how='outer')
x.head(2)

,oldID_x,diagnosis_x,hdrs-total-,dep.severity-,panss-td--,panss-total--,panss-o-total--,panss-p-total--,panss-n-total--,sex_x,...,SOPS Pos Total,SOPS Neg Total,SOPS Des Total,SOPS Com Total,PANSS.ID,panss 1 sum,panss p sum,panss n sum,panss o sum,panss TD
ID,,,,,,,,,,,,,,,,,,,,,
PN-001,H01,NaN,NaN,NaN,4.0,30.0,16.0,7.0,7.0,m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PN-002,H02,NaN,1.0,0.0,6.0,33.0,17.0,8.0,8.0,f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
x['oldID'] = x.apply(lambda row: merge_cols(row, 'oldID_x', 'oldID_y', lambda x, y: x), axis=1)
x.drop(columns=['oldID_x', 'oldID_y'], inplace=True)

In [17]:
x.diagnosis_x = x.diagnosis_x.apply(lambda x: x.replace(' ', '') if not pd.isnull(x) else x)
x['diagnosis'] = x.apply(lambda row: merge_cols(row, 'diagnosis_x', 'diagnosis_y', lambda x, y: x), axis=1)
x.drop(columns=['diagnosis_x', 'diagnosis_y'], inplace=True)

In [18]:
x.loc['PD-115', 'age_x'] = 27.0 
x['age'] = x.apply(lambda row: merge_cols(row, 'age_x', 'age_y', lambda x, y: x), axis=1)
x.drop(columns=['age_x', 'age_y'], inplace=True)

In [19]:
x['sex'] = x.apply(lambda row: merge_cols(row, 'sex_x', 'sex_y', lambda x, y: x), axis=1)
x.drop(columns=['sex_x', 'sex_y'], inplace=True)

In [20]:
x.rename(columns = {'group_x':'group_BA', 'group_y':'group'}, inplace=True)

In [21]:
x['educ.years'] = x.apply(lambda row: merge_cols(row, 'educ.years_x', 'educ.years_y', lambda x, y: x), axis=1)
x.drop(columns=['educ.years_x', 'educ.years_y'], inplace=True)

In [22]:
x['ID'] = x.index
x['ID'] = x.apply(lambda row: merge_cols(row, 'ID', 'PANSS.ID', lambda x, y: x), axis=1)
x['ID'] = x.apply(lambda row: merge_cols(row, 'ID', 'ID.HDRS', lambda x, y: x), axis=1)
x['ID'] = x.apply(lambda row: merge_cols(row, 'ID', 'ID.SOPS', lambda x, y: x), axis=1)
x.drop(columns=['PANSS.ID', 'ID.SOPS', 'ID.HDRS', 'ID'], inplace=True)

In [23]:
x['panss-1-td'] = x.apply(lambda row: merge_cols(row, 'panss-td--', 'panss TD', lambda x, y: x), axis=1)
x.drop(columns=['panss-td--', 'panss TD'], inplace=True)

In [24]:
x.loc['PD-109', 'panss-total--'] = 68.0
x.loc['PD-109', 'panss-o-total--'] = 31.0
x.loc['PD-110', 'panss-total--'] = 93.0
x['panss-1-total'] = x.apply(lambda row: merge_cols(row, 'panss 1 sum', 'panss-total--', lambda x, y: x), axis=1)
x.drop(columns=['panss-total--', 'panss 1 sum'], inplace=True)

In [25]:
x['panss-o-1-total'] = x.apply(lambda row: merge_cols(row, 'panss o sum', 'panss-o-total--', lambda x, y: x), axis=1)
x.drop(columns=['panss-o-total--', 'panss o sum'], inplace=True)

In [26]:
x['panss-p-1-total'] = x.apply(lambda row: merge_cols(row, 'panss p sum', 'panss-p-total--', lambda x, y: x), axis=1)
x.drop(columns=['panss-p-total--', 'panss p sum'], inplace=True)

In [27]:
x['panss-n-1-total'] = x.apply(lambda row: merge_cols(row, 'panss n sum', 'panss-n-total--', lambda x, y: x), axis=1)
x.drop(columns=['panss-n-total--', 'panss n sum'], inplace=True)

In [28]:
x['panss-1-td'] = x.apply(lambda row: merge_cols(row, 'panss-1-td', 'PANSS-FTD', lambda x, y: x), axis=1)
x.drop(columns=['PANSS-FTD'], inplace=True)

In [29]:
x.rename(columns = {'DEP.SEVERITY(0-3)':'dep.severity', 'SOPS Pos Total':'sops-p-total',
                       'SOPS Neg Total': 'sops-n-total', 'SOPS Des Total': 'sops-d-total',
                        'SOPS Com Total': 'sops-c-total', 'TD': 'td.severity', 'SAPS-FTD': 'saps-td',
                       'HDRS': 'hdrs-1-total', 'ds': 'diagnosis_rus', 'Condition-at-discharge':
                       'condition-at-discharge'}, inplace=True)

In [30]:
x['hdrs-1-total']= x.apply(lambda row: merge_cols(row, 'hdrs-total-', 'hdrs-1-total', lambda x, y: x), axis=1)
x.drop(columns=['hdrs-total-'], inplace=True)

In [31]:
x[x['dep.severity'] == 3]['hdrs-1-total']

ID
PD-108    36.0
PD-116    20.0
Name: hdrs-1-total, dtype: float64

In [32]:
x[x['hdrs-1-total'] >= 20][['dep.severity', 'hdrs-1-total']]

,dep.severity,hdrs-1-total
ID,,
PD-108,3.0,36.0
PD-116,3.0,20.0


In [33]:
x.loc['PD-116', 'dep.severity'] = 3.0  # mistake in newer dataset
x['dep.severity'] = x.apply(lambda row: merge_cols(row, 'dep.severity', 'dep.severity-', lambda x, y: x), axis=1)
x.drop(columns=['dep.severity-'], inplace=True)

### merge to PD participant data

In [34]:
x = pd.merge(x, dataframes['PD_participant_data'].set_index('ID'), on='ID', how='outer')
x.head(2)

,group.code,diagnosis_rus,edulevel,socstat1,socstat2,age-onset,GAF1,condition-at-discharge,CPE1,exacerbation,...,sex_y,age_y,education.level,education.years,depression.symptoms,dep.scale,thought.disorder.symptoms,td.scales,2.depression.symptoms,2.thought.disorder.symptoms
ID,,,,,,,,,,,,,,,,,,,,,
PN-001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,male,27.0,higher,22.0,0.0,QIDS,0.0,SCL-90-R,NaN,NaN
PN-002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,female,36.0,higher,17.0,0.0,QIDS,0.0,SCL-90-R,NaN,NaN


In [35]:
x['age'] = x.apply(lambda row: merge_cols(row, 'age_x', 'age_y', lambda x, y: x), axis=1)
x.drop(columns=['age_x', 'age_y'], inplace=True)

In [36]:
x.rename(columns={'diagnosis_x': 'diagnosis_code', 'diagnosis_y': 'diagnosis_eng',
                 '2.depression.symptoms': 'dep.severity-2',
                 '2.thought.disorder.symptoms': 'td.severity-2'}, inplace=True)

In [37]:
x.loc['PD-116', 'depression.symptoms'] = 3.0  # mistake in newer dataset
x['dep.severity'] = x[['depression.symptoms', 'dep.severity']].apply(lambda row: merge_cols(row, 'depression.symptoms', 'dep.severity', lambda x, y: x), axis=1)
x.drop(columns=['depression.symptoms'], inplace=True)

In [38]:
x.loc['PD-126', 'dep.severity']

0.0

In [39]:
x['thought.disorder.symptoms'].dropna()

ID
PN-001    0.0
PN-002    0.0
PN-003    0.0
PN-004    0.0
PN-005    0.0
         ... 
PN-319    0.0
PN-320    0.0
PN-321    0.0
PN-322    0.0
PN-324    0.0
Name: thought.disorder.symptoms, Length: 353, dtype: float64

In [40]:
x[x['saps-td']>=15][['thought.disorder.symptoms', 'td.severity', 'panss-1-td', 'saps-td']]

,thought.disorder.symptoms,td.severity,panss-1-td,saps-td
ID,,,,
PD-050,1.0,2.0,NaN,21.0
PD-092,1.0,1.0,NaN,16.0
PD-095,1.0,1.0,NaN,17.0
PD-093,1.0,1.0,NaN,15.0


In [41]:
x[x['td.severity']>=2][['thought.disorder.symptoms', 'td.severity', 'panss-1-td', 'saps-td']]

,thought.disorder.symptoms,td.severity,panss-1-td,saps-td
ID,,,,
PD-105,2.0,2.0,15.0,NaN
PD-116,3.0,2.0,15.0,NaN
PD-119,2.0,2.0,19.0,NaN
PD-126,2.0,2.0,15.0,NaN
PD-050,1.0,2.0,NaN,21.0


In [42]:
x[x['td.severity']>=2][['thought.disorder.symptoms', 'td.severity', 'panss-1-td', 'saps-td']]

,thought.disorder.symptoms,td.severity,panss-1-td,saps-td
ID,,,,
PD-105,2.0,2.0,15.0,NaN
PD-116,3.0,2.0,15.0,NaN
PD-119,2.0,2.0,19.0,NaN
PD-126,2.0,2.0,15.0,NaN
PD-050,1.0,2.0,NaN,21.0


In [43]:
# x.loc['PD-116', 'thought.disorder.symptoms'] = 2.0  # mistake?
# x.loc['PD-050', 'thought.disorder.symptoms'] = 2.0  # mistake?
# x.loc['PD-098', 'thought.disorder.symptoms'] = 0.0  # mistake?
x['td.severity'] = x.apply(lambda row: merge_cols(row, 'thought.disorder.symptoms', 'td.severity', lambda x, y: x), axis=1)
x.drop(columns=['thought.disorder.symptoms'], inplace=True)

group.code                                        NaN
diagnosis_rus                                     NaN
edulevel                                          NaN
socstat1                                          NaN
socstat2                                          NaN
age-onset                                         NaN
GAF1                                              NaN
condition-at-discharge                            NaN
CPE1                                              NaN
exacerbation                                      NaN
rehospitalization                                 NaN
GAF2                                              NaN
CPE2                                              NaN
group_x                                            sz
diagnosis.rus                                     NaN
dep.severity                                      3.0
hdrs-1-total                                     20.0
td.severity                                       2.0
saps-td                     

In [44]:
x.rename(columns={'edulevel': 'education.level.code', 'education.level': 'education.level.eng'}, inplace=True)

In [45]:
x.loc['PD-148', 'education.years'] = 11.0  # mistake ????
x['education.years'] = x.apply(lambda row: merge_cols(row, 'education.years', 'educ.years', lambda x, y: x), axis=1)
x.drop(columns=['educ.years'], inplace=True)

In [46]:
x.rename(columns={'group_x': 'diagnosis.type', 'group_y': 'diagnosis.group'}, inplace=True)

In [47]:
# rename f to female and m to male
def sex(row):    
    sex_x, sex_y = row['sex_x'], row['sex_y']
    if pd.isnull(sex_x) and pd.isnull(sex_y):
        return sex_x
    elif not pd.isnull(sex_x) and pd.isnull(sex_y):
        return 'female' if sex_x == 'f' else 'male'
    elif pd.isnull(sex_x) and not pd.isnull(sex_y):
        return sex_y
    else:
        if sex_x == sex_y:
            return sex_y
        else:
            if sex_x == 'f' and sex_y == 'female':
                return sex_y
            elif sex_x == 'm' and sex_y == 'male':
                return sex_y
            else:
                print(row)
                return sex_y

In [48]:
x.loc['PN-023', 'sex_x'] = 'f'  # mistake
x['sex'] = x[['sex_x', 'sex_y']].apply(sex, axis=1)
x.drop(columns=['sex_x', 'sex_y'], inplace=True)

In [49]:
cols_test = [c for c in ['diagnosis.type',
 'diagnosis_code',
 'diagnosis_eng',
 'diagnosis_rus',] if c in x.columns] 
if not cols_test:
    cols_test = x.columns
x.loc['PD-019'][cols_test]

diagnosis.type                                dep
diagnosis_code                                F31
diagnosis_eng     borderline.personality.disorder
diagnosis_rus                                 ПРЛ
Name: PD-019, dtype: object

### merge to single-scale totals

In [50]:
x = pd.merge(x, dataframes['sans-1-total'].set_index('ID'), on='ID', how='outer')

In [51]:
x = pd.merge(x, dataframes['sans-2-total'].set_index('ID'), on='ID', how='outer')

In [52]:
x = pd.merge(x, dataframes['hdrs-0-total'].set_index('ID'), on='ID', how='outer')

In [53]:
x = pd.merge(x, dataframes['hdrs-total'].set_index('ID'), on='ID', how='outer')

In [54]:
x['HDRS-17.score'] = x.apply(lambda row: merge_cols(row, 'hdrs-1-total', 'HDRS-17.score', lambda x, y: x), axis=1)
x.drop(columns=['hdrs-1-total'], inplace=True)

In [55]:
x = pd.merge(x, dataframes['hdrs-1-total'].set_index('ID'), on='ID', how='outer')

In [56]:
x = pd.merge(x, dataframes['hdrs-2-total'].set_index('ID'), on='ID', how='outer')

In [57]:
x.drop(columns=['hdrs-0-total',
 'hdrs-1-total',
 'hdrs-2-total',
 'hdrs-total-dep.severity-1',
 'hdrs-total-dep.severity-2',], inplace=True)

In [58]:
x.loc['PD-116', 'dep.severity_y'] = 3.0  # mistake in newer dataset
x['dep.severity-total'] = x.apply(lambda row: merge_cols(row, 'dep.severity_x', 'dep.severity_y', lambda x, y: x), axis=1)
x.drop(columns=['dep.severity_x', 'dep.severity_y'], inplace=True)

#### look for differences in HDRS-17 at different timepoints

In [59]:
compare_hdrs = x[['HDRS-17.score', 'HDRS-17.score-0', 'HDRS-17.score-1', 'HDRS-17.score-2',]]

In [60]:
def compare_cols(row, colname_1, colname_2):
    col_1, col_2 = row[colname_1], row[colname_2]
    if pd.isnull(col_1) and pd.isnull(col_2):
        return True
    elif not pd.isnull(col_1) and pd.isnull(col_2):
        return True
    elif pd.isnull(col_1) and not pd.isnull(col_2):
        return True
    else:
        if col_1 == col_2:
            return True
        else:
            return False

In [61]:
compare_hdrs['comp-0'] = compare_hdrs.apply(lambda row: compare_cols(row, 'HDRS-17.score', 'HDRS-17.score-0'), axis=1)
compare_hdrs['comp-1'] = compare_hdrs.apply(lambda row: compare_cols(row, 'HDRS-17.score', 'HDRS-17.score-1'), axis=1)
compare_hdrs['comp-2'] = compare_hdrs.apply(lambda row: compare_cols(row, 'HDRS-17.score', 'HDRS-17.score-2'), axis=1)

/var/folders/1l/2khv4cgj7xs0zrm9hnl_yrzh0000gn/T/ipykernel_2890/1498707070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare_hdrs['comp-0'] = compare_hdrs.apply(lambda row: compare_cols(row, 'HDRS-17.score', 'HDRS-17.score-0'), axis=1)
/var/folders/1l/2khv4cgj7xs0zrm9hnl_yrzh0000gn/T/ipykernel_2890/1498707070.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare_hdrs['comp-1'] = compare_hdrs.apply(lambda row: compare_cols(row, 'HDRS-17.score', 'HDRS-17.score-1'), axis=1)
/var/folders/1l/2khv

In [62]:
compare_hdrs['comp-0'].value_counts()

True     422
False    114
Name: comp-0, dtype: int64

In [63]:
compare_hdrs['comp-1'].value_counts()

True     427
False    109
Name: comp-1, dtype: int64

In [64]:
compare_hdrs['comp-2'].value_counts()

True     475
False     61
Name: comp-2, dtype: int64

In [65]:
print(compare_hdrs[~compare_hdrs['comp-1']].to_csv(sep='\t'))

ID	HDRS-17.score	HDRS-17.score-0	HDRS-17.score-1	HDRS-17.score-2	comp-0	comp-1	comp-2
PN-003	3.0		0.0		True	False	True
PN-013	1.0		3.0		True	False	True
PD-002	13.0	35.0	0.0		False	False	True
PD-004	15.0	21.0	13.0	10.0	False	False	False
PD-006	5.0	20.0	15.0	12.0	False	False	False
PD-016	15.0	28.0	9.0	14.0	False	False	False
PD-017	5.0	15.0	2.0		False	False	True
PD-018	10.0	23.0	15.0	0.0	False	False	False
PD-020	2.0	16.0	10.0		False	False	True
PD-021	16.0	17.0	17.0	2.0	False	False	False
PD-022	5.0	16.0	2.0		False	False	True
PD-023	10.0	18.0	16.0	8.0	False	False	False
PD-024	14.0	19.0	5.0		False	False	True
PD-025	1.0	19.0	10.0	0.0	False	False	False
PD-029	5.0	12.0	9.0	0.0	False	False	False
PD-031	2.0	25.0	5.0	2.0	False	False	True
PD-032	6.0	19.0	4.0	4.0	False	False	False
PD-033	5.0	16.0	2.0	14.0	False	False	False
PD-034	3.0	25.0	6.0	14.0	False	False	False
PD-035	4.0	17.0	5.0	3.0	False	False	False
PD-037	2.0	23.0	4.0	5.0	False	False	False
PD-042	7.0	16.0	4.0	8.0	False	False	False
PD-047	4.0

In [66]:
x.loc['PD-007', 'dep.severity-2_x'] = 1.0  # mistake
x.loc['PD-033', 'dep.severity-2_x'] = 1.0  # mistake
x.loc['PD-034', 'dep.severity-2_x'] = 1.0  # mistake
x.loc['PD-087', 'dep.severity-2_x'] = 1.0  # mistake
x.loc['PD-059', 'dep.severity-2_x'] = 1.0  # mistake
x.loc['PD-061', 'dep.severity-2_x'] = 1.0  # mistake
x.loc['PD-044', 'dep.severity-2_x'] = 4.0  # mistake
x.loc['PD-046', 'dep.severity-2_y'] = 3.0  # mistake
x.loc['PD-051', 'dep.severity-2_y'] = 3.0  # mistake
x.loc['PD-057', 'dep.severity-2_y'] = 3.0  # mistake
x['dep.severity-2'] = x[['dep.severity-2_x', 'dep.severity-2_y', 'HDRS-17.score-2']].apply(lambda row: merge_cols(row, 'dep.severity-2_x', 'dep.severity-2_y', lambda x, y: x), axis=1)
x.drop(columns=['dep.severity-2_x', 'dep.severity-2_y'], inplace=True)

In [67]:
x[['dep.severity-0',
 'dep.severity-1',
 'dep.severity-2']].dropna(thresh=2)

,dep.severity-0,dep.severity-1,dep.severity-2
ID,,,
PD-002,4.0,0.0,NaN
PD-004,2.0,1.0,1.0
PD-006,2.0,1.0,1.0
PD-007,2.0,0.0,1.0
PD-016,4.0,1.0,1.0
...,...,...,...
PD-141,4.0,0.0,NaN
PD-145,4.0,0.0,NaN
PD-148,4.0,0.0,NaN


In [68]:
# x[['HDRS-17.score', 'HDRS-17.score-1']].apply(lambda row: merge_cols(row, 'HDRS-17.score', 'HDRS-17.score-1', lambda x, y: x), axis=1)

In [69]:
x.loc['PD-126', 'dep.severity-total']

0.0

### merge to PANSS

In [70]:
x = pd.merge(x, dataframes['panss'].set_index('ID'), on='ID', how='outer')

In [71]:
x['panss-o-1-total'] = x.apply(lambda row: merge_cols(row, 'PANSS O', 'panss-o-1-total', lambda x, y: x), axis=1)
x.drop(columns=['PANSS O'], inplace=True)

In [72]:
x['panss-p-1-total'] = x.apply(lambda row: merge_cols(row, 'PANSS P', 'panss-p-1-total', lambda x, y: x), axis=1)
x.drop(columns=['PANSS P'], inplace=True)

In [73]:
x.rename(columns={'PANSS 016': 'PANSS O16', 'PANSS 09': 'PANSS O9'}, inplace=True)

In [74]:
x = pd.merge(x, dataframes['panss-'].set_index('ID'), on='ID', how='outer')

In [75]:
x['panss-n-1-total'] = x.apply(lambda row: merge_cols(row, 'panss-n-total-', 'panss-n-1-total', lambda x, y: x), axis=1)
x.drop(columns=['panss-n-total-'], inplace=True)

In [76]:
x['panss-1-total'] = x.apply(lambda row: merge_cols(row, 'panss-1-total', 'panss-total-', lambda x, y: x), axis=1)
x.drop(columns=['panss-total-'], inplace=True)

In [77]:
x['panss-o-1-total'] = x.apply(lambda row: merge_cols(row, 'panss-o-1-total', 'panss-o-total-', lambda x, y: x), axis=1)
x.drop(columns=['panss-o-total-'], inplace=True)

In [78]:
x['panss-p-1-total'] = x.apply(lambda row: merge_cols(row, 'panss-p-1-total', 'panss-p-total-', lambda x, y: x), axis=1)
x.drop(columns=['panss-p-total-'], inplace=True)

#### identify PANSS inconsistencies

In [79]:
compare_panss_n = pd.DataFrame()

compare_panss_n['PANSS N'] = x['PANSS N']
compare_panss_n['panss-n-1-total'] = x['panss-n-1-total']

compare_panss_n

,PANSS N,panss-n-1-total
ID,,
PN-001,NaN,7.0
PN-002,NaN,8.0
PN-003,NaN,7.0
PN-004,NaN,7.0
PN-005,NaN,9.0
...,...,...
NaN,NaN,NaN
?-1,30.0,NaN
?-2,23.0,NaN


In [80]:
compare_panss_n.apply(lambda row: merge_cols(row, 'PANSS N', 'panss-n-1-total', lambda x, y: x), axis=1)

PANSS N             8.0
panss-n-1-total    10.0
Name: PD-002, dtype: float64
PANSS N            7.0
panss-n-1-total    8.0
Name: PD-006, dtype: float64
PANSS N            7.0
panss-n-1-total    9.0
Name: PD-007, dtype: float64
PANSS N            7.0
panss-n-1-total    8.0
Name: PD-017, dtype: float64
PANSS N             9.0
panss-n-1-total    10.0
Name: PD-018, dtype: float64
PANSS N            7.0
panss-n-1-total    9.0
Name: PD-020, dtype: float64
PANSS N            7.0
panss-n-1-total    9.0
Name: PD-023, dtype: float64
PANSS N            7.0
panss-n-1-total    8.0
Name: PD-025, dtype: float64
PANSS N            7.0
panss-n-1-total    8.0
Name: PD-029, dtype: float64
PANSS N            14.0
panss-n-1-total    17.0
Name: PD-032, dtype: float64
PANSS N            22.0
panss-n-1-total    25.0
Name: PD-042, dtype: float64
PANSS N            13.0
panss-n-1-total    15.0
Name: PD-008, dtype: float64
PANSS N            10.0
panss-n-1-total    12.0
Name: PD-019, dtype: float64
PANSS N      

ID
PN-001     7.0
PN-002     8.0
PN-003     7.0
PN-004     7.0
PN-005     9.0
          ... 
NaN        NaN
?-1       30.0
?-2       23.0
?-3       29.0
?-4       19.0
Length: 540, dtype: float64

In [81]:
compare_panss_n['sum'] = x[[
 'PANSS N1',
 'PANSS N2',
 'PANSS N3',
 'PANSS N4',
 'PANSS N5',
 'PANSS N6',
 'PANSS N7',]].sum(axis=1, min_count=1)
compare_panss_n.apply(lambda row: merge_cols(row, 'PANSS N', 'sum', lambda x, y: x), axis=1)

ID
PN-001     NaN
PN-002     NaN
PN-003     NaN
PN-004     NaN
PN-005     NaN
          ... 
NaN        NaN
?-1       30.0
?-2       23.0
?-3       29.0
?-4       19.0
Length: 540, dtype: float64

In [82]:
# I believe the correct is PANSS N, not panss-n-1-total

In [83]:
compare_panss_total = pd.DataFrame()

In [84]:
compare_panss_total['sum'] = x[['panss-n-1-total', 'panss-o-1-total', 'panss-p-1-total',]].sum(axis=1, min_count=1)
compare_panss_total['sum-'] = x[['PANSS N', 'panss-o-1-total', 'panss-p-1-total',]].sum(axis=1, min_count=1)

In [85]:
compare_panss_total['panss-total'] = x['panss-total']
compare_panss_total['panss-1-total'] = x['panss-1-total']

In [86]:
x['panss-1-td'] = x.apply(lambda row: merge_cols(row, 'panss-td-total', 'panss-1-td', lambda x, y: x), axis=1)
x.drop(columns=['panss-td-total'], inplace=True)

In [87]:
compare_panss_total
# DUE TO DIFFS IN NEG

,sum,sum-,panss-total,panss-1-total
ID,,,,
PN-001,30.0,23.0,NaN,30.0
PN-002,33.0,25.0,NaN,33.0
PN-003,30.0,23.0,NaN,30.0
PN-004,30.0,23.0,NaN,30.0
PN-005,35.0,26.0,NaN,35.0
...,...,...,...,...
NaN,NaN,NaN,NaN,NaN
?-1,46.0,76.0,76.0,NaN
?-2,60.0,83.0,83.0,NaN


In [88]:
dataframes.keys()

dict_keys(['patient_data', 'PD_participant_data', 'combined_old_data_scales', 'demography', 'sans-1-total', 'sans-2-total', 'hdrs-0-total', 'hdrs-total', 'hdrs-1-total', 'hdrs-2-total', 'panss', 'panss-', 'saps-ftd-1-total', 'saps-ftd-2-total', 'sops', 'sops-1-total', 'sops-2-total'])

In [89]:
x['panss-n-1-total'] = x.apply(lambda row: merge_cols(row, 'PANSS N', 'panss-n-1-total', lambda x, y: x), axis=1)
x.drop(columns=['PANSS N'], inplace=True)

group.code              1.0
diagnosis_rus           БАР
education.level.code    4.0
socstat1                2.0
socstat2                NaN
                       ... 
PANSS O12               3.0
PANSS O13               1.0
PANSS O14               1.0
PANSS O15               1.0
PANSS O16               3.0
Name: PD-002, Length: 80, dtype: object
group.code              1.0
diagnosis_rus           ПРЛ
education.level.code    3.0
socstat1                1.0
socstat2                2.0
                       ... 
PANSS O12               1.0
PANSS O13               1.0
PANSS O14               1.0
PANSS O15               1.0
PANSS O16               1.0
Name: PD-006, Length: 80, dtype: object
group.code              1.0
diagnosis_rus           БАР
education.level.code    4.0
socstat1                2.0
socstat2                0.0
                       ... 
PANSS O12               1.0
PANSS O13               1.0
PANSS O14               1.0
PANSS O15               1.0
PANSS O16               

In [90]:
x['panss-sum'] = x[['panss-n-1-total', 'panss-o-1-total', 'panss-p-1-total',]].sum(axis=1, min_count=1)

In [91]:
x['panss-total'] = x.apply(lambda row: merge_cols(row, 'panss-total', 'panss-sum', lambda x, y: x), axis=1)


In [92]:
x['panss-1-total'] = x.apply(lambda row: merge_cols(row, 'panss-total', 'panss-1-total', lambda x, y: x), axis=1)
x.drop(columns=['panss-total'], inplace=True)

group.code               1.0
diagnosis_rus            БАР
education.level.code     4.0
socstat1                 2.0
socstat2                 NaN
                        ... 
PANSS O13                1.0
PANSS O14                1.0
PANSS O15                1.0
PANSS O16                3.0
panss-sum               41.0
Name: PD-002, Length: 80, dtype: object
group.code               1.0
diagnosis_rus            ПРЛ
education.level.code     3.0
socstat1                 1.0
socstat2                 2.0
                        ... 
PANSS O13                1.0
PANSS O14                1.0
PANSS O15                1.0
PANSS O16                1.0
panss-sum               33.0
Name: PD-006, Length: 80, dtype: object
group.code               1.0
diagnosis_rus            БАР
education.level.code     4.0
socstat1                 2.0
socstat2                 0.0
                        ... 
PANSS O13                1.0
PANSS O14                1.0
PANSS O15                1.0
PANSS O16            

### merge to saps and sops

In [93]:
x = pd.merge(x, dataframes['saps-ftd-1-total'].set_index('ID'), on='ID', how='outer')

In [94]:
x['saps-ftd-1-total'] = x.apply(lambda row: merge_cols(row, 'saps-ftd-1-total', 'saps-td', lambda x, y: x), axis=1)
x.drop(columns=['saps-td'], inplace=True)

In [95]:
x = pd.merge(x, dataframes['saps-ftd-2-total'].set_index('ID'), on='ID', how='outer')

In [96]:
x = pd.merge(x, dataframes['sops'].set_index('ID'), on='ID', how='outer')

In [97]:
x['sops-c-total'] = x.apply(lambda row: merge_cols(row, 'sops-c-total', 'sops-com-total', lambda x, y: x), axis=1)
x['sops-d-total'] = x.apply(lambda row: merge_cols(row, 'sops-d-total', 'sops-dep-total', lambda x, y: x), axis=1)
x['sops-n-total'] = x.apply(lambda row: merge_cols(row, 'sops-n-total', 'sops-neg-total', lambda x, y: x), axis=1)
x['sops-p-total'] = x.apply(lambda row: merge_cols(row, 'sops-p-total', 'sops-pos-total', lambda x, y: x), axis=1)
x.drop(columns=['sops-com-total', 'sops-dep-total', 'sops-neg-total', 'sops-pos-total'], inplace=True)

In [98]:
x = pd.merge(x, dataframes['sops-1-total'].set_index('ID'), on='ID', how='outer')
x = pd.merge(x, dataframes['sops-2-total'].set_index('ID'), on='ID', how='outer')

In [99]:
# diagnosis_rus should be correct, as it comes from demograpy datasheet which is newer
x['diagnosis_rus'] = x[['diagnosis_rus', 'diagnosis.rus']].apply(lambda row: merge_cols(row, 'diagnosis_rus', 'diagnosis.rus', lambda x, y: x), axis=1)
x.drop(columns=['diagnosis.rus'], inplace=True)

diagnosis_rus    БАР
diagnosis.rus    РДР
Name: PD-055, dtype: object
diagnosis_rus    БАР
diagnosis.rus    РДР
Name: PD-094, dtype: object
diagnosis_rus                РЛ
diagnosis.rus    Шизотипическое
Name: PD-005, dtype: object
diagnosis_rus    Шизотипическое
diagnosis.rus        Шизофрения
Name: PD-040, dtype: object
diagnosis_rus               БАР
diagnosis.rus    Шизотипическое
Name: PD-043, dtype: object
diagnosis_rus               БАР
diagnosis.rus    Шизотипическое
Name: PD-070, dtype: object
diagnosis_rus               БАР
diagnosis.rus    Шизотипическое
Name: PD-077, dtype: object
diagnosis_rus    РДР
diagnosis.rus    ШАР
Name: PD-139, dtype: object


In [100]:
# sorted(x.columns, key=lambda x: x.lower())

## identify NAs

In [101]:
cols = ['age',
 'dep.scale',
 'dep.severity-1',
 'group.code',
 'diagnosis.group',
 'diagnosis.type',
 'diagnosis_code',
 'diagnosis_eng',
 'diagnosis_rus',
 'education.level.code',
 'education.level.eng',
 'education.years',
 'exacerbation',
#  'HDRS-17.score',
 'HDRS-17.score-1',
 'longitudinal',
#  'PANSS N',
 'panss-1-td',
 'panss-1-total',
 'panss-n-1-total',
 'panss-o-1-total',
 'panss-p-1-total',
#  'panss-total',
 'rehospitalization',
 'sans-1-total',
 'saps-ftd-1-total',
 'sex',
 'sops-1-total',
 'sops-c-total',
 'sops-d-total',
 'sops-n-total',
 'sops-p-total',
 'td.scales',
 'td.severity']

In [102]:
x = x[~x.index.isna()]

### fill NA in diagnosis

In [103]:
def fill_group_na(row):
    dg = row['diagnosis.group']
    if not pd.isnull(dg):
        return dg
    elif row.name.startswith('PN'):
        return 'control'
    elif row.name.startswith('PD'):
        return 'patient'
    else:
        return np.nan

In [104]:
x['diagnosis.group'] = x.apply(fill_group_na, axis=1)

In [105]:
len(x)

445

In [106]:
x[cols].isna().sum(axis=0)

age                      58
dep.scale                81
dep.severity-1          274
group.code              303
diagnosis.group           4
diagnosis.type          356
diagnosis_code          330
diagnosis_eng           292
diagnosis_rus           303
education.level.code    303
education.level.eng      81
education.years          57
exacerbation            337
HDRS-17.score-1         274
longitudinal            382
panss-1-td              333
panss-1-total           326
panss-n-1-total         326
panss-o-1-total         326
panss-p-1-total         326
rehospitalization       332
sans-1-total            305
saps-ftd-1-total        305
sex                      74
sops-1-total            436
sops-c-total            424
sops-d-total            424
sops-n-total            424
sops-p-total            424
td.scales                81
td.severity              78
dtype: int64

In [107]:
len(x[x['diagnosis.group'] == 'patient'])

227

In [108]:
x[x['diagnosis.group'] == 'patient'][['diagnosis.type',
 'diagnosis_code',
 'diagnosis_eng',
 'diagnosis_rus',]]

,diagnosis.type,diagnosis_code,diagnosis_eng,diagnosis_rus
ID,,,,
PD-101,sz,F20,schizophrenia,NaN
PD-102,sz,F20,schizophrenia,NaN
PD-103,sz,F20,schizophrenia,NaN
PD-105,sz,F20,schizophrenia,NaN
PD-106,sz,F20,schizophrenia,NaN
...,...,...,...,...
PD-196,NaN,NaN,NaN,NaN
PD-197,NaN,NaN,NaN,NaN
PD-198,NaN,NaN,NaN,NaN


In [109]:
x['diagnosis.type'].value_counts()

dep    54
sz     35
Name: diagnosis.type, dtype: int64

In [110]:
x['diagnosis_code'].value_counts()

F20      27
F31      18
F25      15
F31.3    11
F21?      5
F60       5
F21       4
F31.5     4
F33       4
F21.3     3
F32       3
F32.0     3
F32.1     3
F31.4     3
F34.1     2
F41.2     1
F33.3     1
F32.3     1
F23       1
F33.1     1
Name: diagnosis_code, dtype: int64

In [111]:
x['diagnosis_eng'].value_counts()

bipolar.affective.disorder         39
schizophrenia                      37
schizotypal.disorder               22
schizoaffective.disorder           17
borderline.personality.disorder    15
recurrent.depressive.disorder      13
depressive.episode                  5
personality.disorder                5
Name: diagnosis_eng, dtype: int64

In [112]:
x['diagnosis_rus'].value_counts()

БАР               45
Шизофрения        23
Шизотипическое    21
ПРЛ               17
РДР               14
ШАР               10
РЛ                 7
Первый             5
Name: diagnosis_rus, dtype: int64

In [113]:
diagnosis_rus_eng = {
                    'БАР': 'bipolar.affective.disorder',
                    'Шизофрения': 'schizophrenia',
                    'Шизотипическое': 'schizotypal.disorder',
                    'ПРЛ': 'borderline.personality.disorder',
                    'РДР': 'recurrent.depressive.disorder',
                    'ШАР': 'schizoaffective.disorder',
                    'РЛ': 'personality.disorder', 
                    'Первый': 'depressive.episode'}

In [114]:
diagnosis_eng_rus = {v: k for k, v in diagnosis_rus_eng.items()}

In [115]:
diagnosis_code_eng = {
'F20': 'schizophrenia', 
'F21': 'schizotypal.disorder', 
'F21.3': 'schizotypal.disorder.pseudoneurotic.schizophrenia', 
'F23': 'acute.psychotic.disorder', 
'F25': 'schizoaffective.disorder', 
'F31': 'bipolar.affective.disorder', 
'F31.3': 'bipolar.affective.disorder.mild', 
'F31.4': 'bipolar.affective.disorder.severe',
'F31.5': 'bipolar.affective.disorder.severe.psychotic', 
'F32': 'depressive.episode', 
'F32.0': 'depressive.episode.mild', 
'F32.1': 'depressive.episode.moderate', 
'F32.3': 'depressive.episode.severe.psychotic',
'F33': 'recurrent.depressive.disorder', 
'F33.1': 'recurrent.depressive.disorder.moderate',
'F33.3': 'recurrent.depressive.disorder.severe.psychotic',
'F34.1': 'dysthymia',
'F41.2': 'mixed.anxiety.depressive.disorder',
'F60': 'personality.disorder', 
'F60.31': 'borderline.personality.disorder',
'F21?': 'exclude: suspected.schizotypal.disorder'
}

In [116]:
diagnosis_code_type = {
'F20': 'sz', 
'F21': 'sz', 
'F21.3': 'sz', 
'F23': 'sz', 
'F25': 'sz', 
'F31': 'dep', 
'F31.3': 'dep', 
'F31.4': 'dep',
'F31.5': 'dep', 
'F32': 'dep', 
'F32.0': 'dep', 
'F32.1': 'dep', 
'F32.3': 'dep',
'F33': 'dep', 
'F33.1': 'dep',
'F33.3': 'dep',
'F34.1': 'dep',
'F41.2': 'dep',
'F60': 'dep', 
'F60.31': 'dep',
'F21?': 'exclude',
}

In [117]:
diagnosis_eng_code = {v: k for k, v in diagnosis_code_eng.items()}

In [118]:
def fill_diagnosis(row):
    diagnosis_code, diagnosis_rus, diagnosis_eng = row['diagnosis_code'], row['diagnosis_rus'], row['diagnosis_eng']
    if pd.isnull(diagnosis_code) and pd.isnull(diagnosis_eng) and pd.isnull(diagnosis_rus):
        if row['diagnosis.type'] == 'patient':
            print(row.name)
        # all are nan
        
    elif not pd.isnull(diagnosis_rus):
        # diagnosis_rus is the most reliable
        diagnosis_eng = diagnosis_rus_eng[diagnosis_rus]
    
    if not pd.isnull(diagnosis_code) and pd.isnull(diagnosis_eng):
        # code but no eng
        diagnosis_eng = diagnosis_code_eng[diagnosis_code]
        
    elif pd.isnull(diagnosis_code) and not pd.isnull(diagnosis_eng):
        # eng but no code
        diagnosis_code = diagnosis_eng_code[diagnosis_eng]
    
    elif not pd.isnull(diagnosis_code) and not pd.isnull(diagnosis_eng):
        # both
        eng = diagnosis_code_eng[diagnosis_code]
        code = diagnosis_eng_code[diagnosis_eng]
        if code != diagnosis_code or eng != diagnosis_eng:
            print(row.name, code, diagnosis_code, eng, diagnosis_eng)
        diagnosis_eng = eng if diagnosis_code.startswith(code) else diagnosis_eng
        diagnosis_code = diagnosis_code if diagnosis_code.startswith(code) else code
        if code != diagnosis_code or eng != diagnosis_eng:
            print(row.name, diagnosis_code, diagnosis_eng)
    return diagnosis_code, diagnosis_eng
        
        

In [119]:
x[['diagnosis_code', 'diagnosis_eng']] = pd.DataFrame(x.apply(fill_diagnosis, axis=1).tolist(), index=x.index, columns=['diagnosis_code', 'diagnosis_eng'])

PD-002 F31 F31.4 bipolar.affective.disorder.severe bipolar.affective.disorder
PD-002 F31.4 bipolar.affective.disorder.severe
PD-004 F60.31 F60 personality.disorder borderline.personality.disorder
PD-004 F60.31 borderline.personality.disorder
PD-006 F60.31 F60 personality.disorder borderline.personality.disorder
PD-006 F60.31 borderline.personality.disorder
PD-016 F31 F31.4 bipolar.affective.disorder.severe bipolar.affective.disorder
PD-016 F31.4 bipolar.affective.disorder.severe
PD-020 F32 F32.1 depressive.episode.moderate depressive.episode
PD-020 F32.1 depressive.episode.moderate
PD-021 F33 F32.1 depressive.episode.moderate recurrent.depressive.disorder
PD-021 F33 recurrent.depressive.disorder
PD-022 F31 F31.4 bipolar.affective.disorder.severe bipolar.affective.disorder
PD-022 F31.4 bipolar.affective.disorder.severe
PD-023 F31 F32.0 depressive.episode.mild bipolar.affective.disorder
PD-023 F31 bipolar.affective.disorder
PD-024 F33 F31 bipolar.affective.disorder recurrent.depressive.d

In [120]:
x[x['diagnosis.group'] == 'patient'][['diagnosis.type',
 'diagnosis_code',
 'diagnosis_eng',
 'diagnosis_rus',]]

,diagnosis.type,diagnosis_code,diagnosis_eng,diagnosis_rus
ID,,,,
PD-101,sz,F20,schizophrenia,NaN
PD-102,sz,F20,schizophrenia,NaN
PD-103,sz,F20,schizophrenia,NaN
PD-105,sz,F20,schizophrenia,NaN
PD-106,sz,F20,schizophrenia,NaN
...,...,...,...,...
PD-196,NaN,NaN,NaN,NaN
PD-197,NaN,NaN,NaN,NaN
PD-198,NaN,NaN,NaN,NaN


In [121]:
def fill_diagnosis_group(row):
    dg = row['diagnosis.type']
    code = row['diagnosis_code']
    if pd.isnull(dg) and pd.isnull(code):
        return np.nan
    elif pd.isnull(dg) and not pd.isnull(code):
        return diagnosis_code_type[code]
    elif not pd.isnull(dg) and not pd.isnull(code):
        if not diagnosis_code_type[code] == dg:
            print(row.name, dg, diagnosis_code_type[code])
        return diagnosis_code_type[code]

In [122]:
x['diagnosis.type'] = x.apply(fill_diagnosis_group, axis=1)

PD-003 dep sz
PD-005 sz dep
PD-015 dep sz
PD-044 dep sz


In [123]:
x[x['diagnosis.group'] == 'patient'][cols].isna().sum(axis=0)

age                      53
dep.scale                74
dep.severity-1           82
group.code               85
diagnosis.group           0
diagnosis.type           53
diagnosis_code           53
diagnosis_eng            53
diagnosis_rus            85
education.level.code     85
education.level.eng      74
education.years          53
exacerbation            119
HDRS-17.score-1          82
longitudinal            164
panss-1-td              146
panss-1-total           143
panss-n-1-total         143
panss-o-1-total         143
panss-p-1-total         143
rehospitalization       114
sans-1-total             87
saps-ftd-1-total         87
sex                      70
sops-1-total            218
sops-c-total            206
sops-d-total            206
sops-n-total            206
sops-p-total            206
td.scales                74
td.severity              71
dtype: int64

### identifty NA in controls

In [124]:
control_cols = ['age',
 'dep.scale',
 'dep.severity-1',
#  'dep.severity-total',
 'education.level.eng',
 'education.years',
#  'HDRS-17.score',
 'HDRS-17.score-1',
#  'PANSS N',
 'panss-1-td',
 'panss-1-total',
 'panss-n-1-total',
 'panss-o-1-total',
 'panss-p-1-total',
#  'panss-total',
 'sex',
 'td.scales',
 'td.severity']

x[x['diagnosis.group'] == 'control'][control_cols].isna().sum(axis=0)

age                      1
dep.scale                3
dep.severity-1         188
education.level.eng      3
education.years          0
HDRS-17.score-1        188
panss-1-td             183
panss-1-total          183
panss-n-1-total        183
panss-o-1-total        183
panss-p-1-total        183
sex                      0
td.scales                3
td.severity              3
dtype: int64

In [125]:
len(x[x['diagnosis.group'] == 'control'])

214

### Add self-reported norm data

In [126]:
norm_demography = pd.read_csv('/Users/galina.ryazanskaya/Downloads/thesis?/2023_annotation/norm.sbj.csv', index_col=0)

In [127]:
norm_demography.index

Index(['N-01', 'N-02', 'N-03', 'N-04', 'N-05', 'N-06', 'N-07', 'N-30', 'N-09',
       'N-10', 'N-11', 'N-12', 'N-13', 'N-14', 'N-15', 'N-16', 'N-17', 'N-18',
       'N-19', 'N-20', 'N-21', 'N-22', 'N-23', 'N-24', 'N-25', 'N-26', 'N-28',
       'N-29', 'N-40', 'N-46', 'N-32', 'N-33', 'N-34', 'N-35', 'N-36', 'N-37',
       'N-38', 'N-51', 'N-54', 'N-41', 'N-42', 'N-43', 'N-44', 'N-45', 'N-55',
       'N-56', 'N-48', 'N-49', 'N-50', 'N-57', 'N-52', 'N-53', 'N-63', 'N-67',
       'N-72', 'N-76', 'N-58', 'N-59', 'N-60', 'N-78', 'N-08', 'N-31', 'N-39',
       'N-47', 'N-69', 'N-70', 'N-71', 'N-61', 'N-73', 'N-74', 'N-75', 'N-62',
       'N-77', 'N-68', 'N-79', 'N-80', 'N-81', 'N-82', 'N-83', 'N-84', 'N-85',
       'N-88', 'N-90', 'N-91', 'N-94', 'N-95'],
      dtype='object', name='ID')

In [128]:
x = pd.merge(x, norm_demography, on='ID', how='outer')

In [129]:
x['age'] = x.apply(lambda row: merge_cols(row, 'age_x', 'age_y', lambda x, y: x), axis=1)
x.drop(columns=['age_x', 'age_y'], inplace=True)

In [130]:
x['education.years'] = x.apply(lambda row: merge_cols(row, 'education.years_x', 'education.years_y', lambda x, y: x), axis=1)
x.drop(columns=['education.years_x', 'education.years_y'], inplace=True)

In [131]:
x['education.level.eng'] = x.apply(lambda row: merge_cols(row, 'education.level.eng', 'education.level', lambda x, y: x), axis=1)
x.drop(columns=['education.level'], inplace=True)

In [132]:
x['sex'] = x.apply(lambda row: merge_cols(row, 'sex_x', 'sex_y', lambda x, y: x), axis=1)
x.drop(columns=['sex_x', 'sex_y'], inplace=True)

In [133]:
x.loc[norm_demography.index, 'diagnosis.group'] = 'control'

In [134]:
sorted(x.columns, key=lambda x: x.lower())

['age',
 'age-onset',
 'age.group',
 'condition-at-discharge',
 'CPE1',
 'CPE2',
 'dep.scale',
 'dep.severity-0',
 'dep.severity-1',
 'dep.severity-2',
 'dep.severity-total',
 'diagnosis.group',
 'diagnosis.type',
 'diagnosis_code',
 'diagnosis_eng',
 'diagnosis_rus',
 'education.level.code',
 'education.level.eng',
 'education.years',
 'exacerbation',
 'GAF1',
 'GAF2',
 'group.code',
 'HDRS-17.score',
 'HDRS-17.score-0',
 'HDRS-17.score-1',
 'HDRS-17.score-2',
 'longitudinal',
 'oldID',
 'PANSS N1',
 'PANSS N2',
 'PANSS N3',
 'PANSS N4',
 'PANSS N5',
 'PANSS N6',
 'PANSS N7',
 'PANSS O1',
 'PANSS O10',
 'PANSS O11',
 'PANSS O12',
 'PANSS O13',
 'PANSS O14',
 'PANSS O15',
 'PANSS O16',
 'PANSS O2',
 'PANSS O3',
 'PANSS O4',
 'PANSS O5',
 'PANSS O6',
 'PANSS O7',
 'PANSS O8',
 'PANSS O9',
 'PANSS P1',
 'PANSS P2',
 'PANSS P3',
 'PANSS P4',
 'PANSS P5',
 'PANSS P6',
 'PANSS P7',
 'panss-1-td',
 'panss-1-total',
 'panss-n-1-total',
 'panss-o-1-total',
 'panss-p-1-total',
 'panss-sum',
 'r

In [135]:
# I believe the correct is PANSS N, not panss-n-1-total
# x.drop(columns=['panss-n-1-total', 'panss-1-total'], inplace=True)
# x.rename(columns={'PANSS N': 'panss-n-1-total', 'panss-total': 'panss-1-total'}, inplace=True)

### infer missing panss td severity

In [136]:
x[x['td.severity'].isna()]['panss-1-td'].dropna()

ID
PN-020    4.0
PN-021    4.0
PN-029    4.0
Name: panss-1-td, dtype: float64

In [137]:
x.loc[x[x['panss-1-td'] < 5]['td.severity'].index, 'td.severity'] = 0.0

In [138]:
x.loc['PN-020']['td.severity']

0.0

### infer missing saps-ftd td.severity

In [139]:
x[x['td.severity'].isna() & ~x['saps-ftd-1-total'].isna()][['td.severity', 'saps-ftd-1-total', 'panss-1-td']]

,td.severity,saps-ftd-1-total,panss-1-td
ID,,,
PD-899,NaN,9.0,NaN
PD-014,NaN,7.0,NaN
PD-069,NaN,9.0,NaN
PD-077,NaN,8.0,NaN
PD-995,NaN,8.0,NaN
PD-998,NaN,9.0,NaN
PD-135,NaN,5.0,NaN
PD-893,NaN,16.0,NaN
PD-894,NaN,18.0,NaN


### infer missing dep.scale

In [140]:
dep_scales = ['dep.scale', 'dep.severity-1', 'HDRS-17.score-1', 'dep.severity-total', 'HDRS-17.score', 'dep.severity-0', 'HDRS-17.score-0']
x[x['dep.scale'].isna() & ~x['dep.severity-1'].isna()][dep_scales]

,dep.scale,dep.severity-1,HDRS-17.score-1,dep.severity-total,HDRS-17.score,dep.severity-0,HDRS-17.score-0
ID,,,,,,,
PN-020,NaN,0.0,1.0,0.0,1.0,NaN,NaN
PN-021,NaN,0.0,3.0,0.0,3.0,NaN,NaN
PN-029,NaN,0.0,0.0,0.0,0.0,NaN,NaN
PD-062,NaN,1.0,12.0,0.0,5.0,2.0,23.0
PD-899,NaN,0.0,5.0,NaN,NaN,2.0,20.0
PD-014,NaN,0.0,2.0,1.0,9.0,1.0,16.0
PD-069,NaN,0.0,6.0,0.0,4.0,2.0,19.0
PD-077,NaN,0.0,4.0,0.0,4.0,2.0,17.0
PD-135,NaN,0.0,4.0,0.0,5.0,4.0,25.0


In [141]:
x[x['dep.scale'].isna() & ~x['dep.severity-total'].isna()][dep_scales]

,dep.scale,dep.severity-1,HDRS-17.score-1,dep.severity-total,HDRS-17.score,dep.severity-0,HDRS-17.score-0
ID,,,,,,,
PN-020,NaN,0.0,1.0,0.0,1.0,NaN,NaN
PN-021,NaN,0.0,3.0,0.0,3.0,NaN,NaN
PN-029,NaN,0.0,0.0,0.0,0.0,NaN,NaN
PD-133,NaN,NaN,NaN,0.0,6.0,NaN,NaN
PD-134,NaN,NaN,NaN,1.0,8.0,NaN,NaN
PD-062,NaN,1.0,12.0,0.0,5.0,2.0,23.0
PD-014,NaN,0.0,2.0,1.0,9.0,1.0,16.0
PD-069,NaN,0.0,6.0,0.0,4.0,2.0,19.0
PD-077,NaN,0.0,4.0,0.0,4.0,2.0,17.0


In [142]:
x.loc[x[x['dep.scale'].isna() & ~x['dep.severity-1'].isna()].index, 'dep.scale'] = 'HDRS'

In [143]:
x.loc[x[x['dep.scale'].isna() & ~x['dep.severity-total'].isna()].index, 'dep.scale'] = 'HDRS'

In [144]:
x[~x['HDRS-17.score-1'].isna() & x['dep.severity-1'].isna()][dep_scales]

,dep.scale,dep.severity-1,HDRS-17.score-1,dep.severity-total,HDRS-17.score,dep.severity-0,HDRS-17.score-0
ID,,,,,,,


In [145]:
x[~x['HDRS-17.score'].isna() & x['dep.severity-total'].isna()][dep_scales]

,dep.scale,dep.severity-1,HDRS-17.score-1,dep.severity-total,HDRS-17.score,dep.severity-0,HDRS-17.score-0
ID,,,,,,,


In [146]:
def score_hdrs_17_severity(score):
    if score <= 7:
        res = 0
    elif score <= 16:
        res = 1
    elif score <= 19:
        res = 2
    elif score <= 23:
        res = 3
    else:
        res = 4
    return res

In [147]:
x[x['HDRS-17.score-1'] >= 19][dep_scales].dropna()

,dep.scale,dep.severity-1,HDRS-17.score-1,dep.severity-total,HDRS-17.score,dep.severity-0,HDRS-17.score-0
ID,,,,,,,


In [148]:
x.loc['PD-116', 'dep.severity-1'] = 3.0

In [149]:
# x.loc[x[~x['HDRS-17.score'].isna() & x['dep.severity-1'].isna()].index, 'HDRS-17.score'] = x[~x['HDRS-17.score'].isna() & x['dep.severity-1'].isna()]['HDRS-17.score'].apply(score_hdrs_17_severity)

In [150]:
x[~x['HDRS-17.score'].isna() & x['HDRS-17.score-1'].isna()][dep_scales]

,dep.scale,dep.severity-1,HDRS-17.score-1,dep.severity-total,HDRS-17.score,dep.severity-0,HDRS-17.score-0
ID,,,,,,,
PD-108,HDRS,NaN,NaN,3.0,36.0,NaN,NaN
PD-129,HDRS,NaN,NaN,1.0,12.0,NaN,NaN
PD-133,HDRS,NaN,NaN,0.0,6.0,NaN,NaN
PD-134,HDRS,NaN,NaN,1.0,8.0,NaN,NaN


In [151]:
x.loc[x[~x['HDRS-17.score'].isna() & x['HDRS-17.score-1'].isna()].index, 'HDRS-17.score-1'] = x[~x['HDRS-17.score'].isna() & x['HDRS-17.score-1'].isna()]['HDRS-17.score']

In [152]:
x.loc[x[~x['dep.severity-total'].isna() & x['dep.severity-1'].isna()].index, 'dep.severity-1'] = x[~x['dep.severity-total'].isna() & x['dep.severity-1'].isna()]['dep.severity-total']

In [153]:
x.loc['PD-126', 'dep.severity-total']

0.0

In [154]:
x.loc['PD-126', 'dep.severity-1']

0.0

In [155]:
x.loc['PD-126', 'HDRS-17.score']

nan

In [156]:
x.loc['PD-126', 'HDRS-17.score-1']

nan

In [157]:
x.loc['PD-126', 'HDRS-17.score-0']

nan

In [158]:
x.loc['PD-126', 'HDRS-17.score-1']

nan

In [159]:
x.drop(columns=['HDRS-17.score', 'dep.severity-total'], inplace=True)

In [160]:
sorted(x.columns)

['CPE1',
 'CPE2',
 'GAF1',
 'GAF2',
 'HDRS-17.score-0',
 'HDRS-17.score-1',
 'HDRS-17.score-2',
 'PANSS N1',
 'PANSS N2',
 'PANSS N3',
 'PANSS N4',
 'PANSS N5',
 'PANSS N6',
 'PANSS N7',
 'PANSS O1',
 'PANSS O10',
 'PANSS O11',
 'PANSS O12',
 'PANSS O13',
 'PANSS O14',
 'PANSS O15',
 'PANSS O16',
 'PANSS O2',
 'PANSS O3',
 'PANSS O4',
 'PANSS O5',
 'PANSS O6',
 'PANSS O7',
 'PANSS O8',
 'PANSS O9',
 'PANSS P1',
 'PANSS P2',
 'PANSS P3',
 'PANSS P4',
 'PANSS P5',
 'PANSS P6',
 'PANSS P7',
 'age',
 'age-onset',
 'age.group',
 'condition-at-discharge',
 'dep.scale',
 'dep.severity-0',
 'dep.severity-1',
 'dep.severity-2',
 'diagnosis.group',
 'diagnosis.type',
 'diagnosis_code',
 'diagnosis_eng',
 'diagnosis_rus',
 'education.level.code',
 'education.level.eng',
 'education.years',
 'exacerbation',
 'group.code',
 'longitudinal',
 'oldID',
 'panss-1-td',
 'panss-1-total',
 'panss-n-1-total',
 'panss-o-1-total',
 'panss-p-1-total',
 'panss-sum',
 'rehospitalization',
 'sans-1-total',
 'san

### save data

In [161]:
x.to_csv('combined_data.tsv', sep='\t')

## Get ids for which transcripts are available

In [163]:
df = pd.read_csv('rus_transcript_lex_by_task_with_dots.tsv', sep='\t', index_col=0)

In [164]:
first = [x.split('_')[0] for x in df.index if x.endswith('1')]
second = [x.split('_')[0] for x in df.index if x.endswith('2')]
third = [x.split('_')[0] for x in df.index if x.endswith('3')]

In [165]:
len(first)

183

In [166]:
len(second)

0

In [167]:
len(third)

0

In [168]:
def change_id(ID):
    if ID.startswith('PD'):
        ID = 'PD-' + ID[2:]
    elif ID.startswith('PN'):
        ID ='PN-' + ID[2:]
    elif ID.startswith('S'):
        ID = 'PD-1' + ID[1:]
    elif ID.startswith('H'):
        ID = 'PN-0' + ID[1:]
    elif ID.startswith('N'):
        ID = 'N-' + ID[1:]
    return ID

In [169]:
used_id_1 = [change_id(i) for i in first]
used_id_2 = [change_id(i) for i in second]
used_id_3 = [change_id(i) for i in third]

## Identify NAs in the used ids

In [170]:
set(used_id_1).difference(set(x.index))

{'PN-238', 'PN-327'}

In [171]:
used_id = set(used_id_1) - set(used_id_1).difference(set(x.index))

In [172]:
x_to_use = x.loc[list(used_id)]

### Drop NA / excluded

In [ ]:
x_to_use.drop(index=x[x['diagnosis_code'] == 'F21?'].index, inplace=True)

In [ ]:
x_to_use.dropna(how='all', inplace=True)

In [ ]:
x_to_use[x_to_use['age'].isna() & x_to_use['sex'].isna()][cols]

In [ ]:
x_to_use.drop(index=x_to_use[x_to_use['age'].isna() & x_to_use['sex'].isna()].index, inplace=True)

In [ ]:
x_to_use[x_to_use['diagnosis.group'] == 'patient'][cols].isna().sum(axis=0)

In [ ]:
x_to_use[x_to_use['diagnosis.group'] == 'patient'][x_to_use[x_to_use['diagnosis.group'] == 'patient']['HDRS-17.score-1'].isna()].index

In [ ]:
cols_for_na_check = ['age', 'sex', 'education.years', 'td.severity', 'diagnosis_code', 'dep.scale', 'dep.severity-1']

In [ ]:
patient_na = x_to_use[x_to_use['diagnosis.group'] == 'patient'][cols_for_na_check][x_to_use[x_to_use['diagnosis.group'] == 'patient'][cols_for_na_check].isna().sum(axis=1) != 0]
patient_na.style.highlight_null()

In [ ]:
patient_na.dropna(how='all').style.highlight_null()

In [ ]:
x_to_use[x_to_use['diagnosis.group'] == 'control'][control_cols].isna().sum(axis=0)

In [ ]:
cols_for_na_check_c = ['age', 'sex', 'education.years', 'td.severity', 'dep.scale', 'dep.severity-1']
control_na = x_to_use[x_to_use['diagnosis.group'] == 'control'][cols_for_na_check_c][x_to_use[x_to_use['diagnosis.group'] == 'control'][cols_for_na_check_c].isna().sum(axis=1) != 0]
control_na.style.highlight_null()

In [ ]:
x_to_use[cols].to_csv('merged_psychosocial_data.csv')